transformer出发，里面的attention、fnn、pe、layernorm、relu函数，各自都有改进。

学习目标：

1. transformer相关
- pe
- attn: se\mha
- fnn
- norm:layernorm\batchnorm
- Relu
- decoder only

2. llama/其他改进
- atte:GQA/MQA/flashAtten
- deepseek：MoE
- RoPE/YaRN
- SwiGLU

3. minimind构建post train
- LoRA
- RL:DPO/PPO
- vLLM：kv cache

# 0. embedding与位置编码相关
- emd【tokenizer部分写】
$$\text{Input} \xrightarrow{\text{分词}} \text{Token} \xrightarrow{\text{Vocabulary}} \text{Token ID} \xrightarrow{\text{Embedding}} \mathbf{X}_{(\text{emd})} \xrightarrow{\oplus \text{PE}} \mathbf{X}_{\text{final}}$$


- PE $seqlen \times d_{model}$
$$PE_{(pos, 2i)} = \sin\left(\frac{pos}{10000^{2i/d_{model}}}\right)$$
$$PE_{(pos, 2i+1)} = \cos\left(\frac{pos}{10000^{2i/d_{model}}}\right)$$


div处理

$$\text{before} = \frac{1}{10000^{2i / d_{model}}}$$

$$\text{after} = e^{\left( - \frac{2i}{d_{model}} \times \ln(10000) \right)}$$

In [4]:
import torch
import math

d_model = 100
max_len = 5000

# i 取range

i = torch.arange(0,d_model,2) # [s,e,step]
 
div = torch.exp(i.float()*(-1)*(math.log(10000)/d_model)) # log底数默认为e，math.log(x, [base])

print(i.shape)
print(div)


# input理解
d_model = 4
seq_len = 3  # 比如句子是 "I love you"
batch_size = 1

x = torch.randn(batch_size, seq_len, d_model)
print(f"输入 x 的 Shape:   {x.shape}")

pe = torch.zeros(1, max_len, d_model)

# 4. 执行切片
sliced_pe = pe[:, :x.size(1), :]

print("-" * 30)
print(f"切出来的 PE Shape:  {sliced_pe.shape}")
print("-" * 30)

# 奇偶理解

indices = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

# 1. 取偶数位 (0::2)
evens = indices[0::2]
print(f"偶数位 (0::2): {evens}")
# 输出: [0, 2, 4, 6, 8]

# 2. 取奇数位 (1::2)
odds = indices[1::2]
print(f"奇数位 (1::2): {odds}")
# 输出: [1, 3, 5, 7, 9]



torch.Size([50])
tensor([1.0000e+00, 8.3176e-01, 6.9183e-01, 5.7544e-01, 4.7863e-01, 3.9811e-01,
        3.3113e-01, 2.7542e-01, 2.2909e-01, 1.9055e-01, 1.5849e-01, 1.3183e-01,
        1.0965e-01, 9.1201e-02, 7.5858e-02, 6.3096e-02, 5.2481e-02, 4.3652e-02,
        3.6308e-02, 3.0200e-02, 2.5119e-02, 2.0893e-02, 1.7378e-02, 1.4454e-02,
        1.2023e-02, 1.0000e-02, 8.3176e-03, 6.9183e-03, 5.7544e-03, 4.7863e-03,
        3.9811e-03, 3.3113e-03, 2.7542e-03, 2.2909e-03, 1.9055e-03, 1.5849e-03,
        1.3183e-03, 1.0965e-03, 9.1201e-04, 7.5858e-04, 6.3096e-04, 5.2481e-04,
        4.3652e-04, 3.6308e-04, 3.0200e-04, 2.5119e-04, 2.0893e-04, 1.7378e-04,
        1.4454e-04, 1.2023e-04])
输入 x 的 Shape:   torch.Size([1, 3, 4])
------------------------------
切出来的 PE Shape:  torch.Size([1, 3, 4])
------------------------------
偶数位 (0::2): [0, 2, 4, 6, 8]
奇数位 (1::2): [1, 3, 5, 7, 9]


In [ ]:
import torch
import torch.nn as nn
import math


class PE(nn.Module):
    
    def __init__(self, d_model ,max_len = 5000):
        super().__init__()
        
        # 1. pe init [max_len,d_model]
        pe = torch.zeros(max_len, d_model)
        # 2. pos [maxlen,1] 【后续YaRN改进motivation，超过max_len未见过】
        position = torch.arange(0, max_len).float().unsqueeze(1)  
    
        # 3. div【token内部d_model做高低频】
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model)) # [d_model/2]
        
        # 4. mul,对d_model维度分奇偶处理
        pe[:, 0::2] = torch.sin(position * div_term) 
        pe[:, 1::2] = torch.cos(position * div_term) 
        
        
        self.register_buffer('pe',pe.unsqueeze(0)) # [batch,seq_len,hidden_dim]
        
    
    
    def forward(self,x):
        #  x shape [batch,seq_len,hidden_dim]
    
        # x = x + position
        
        return x + self.pe[: , :x.size(1) , :] # 对做好的buffer pe 拿出seq_len

d_model = 512
max_len = 5000
seq_len = 64
batch_size = 4

# 初始化模型
pe_layer = PE(d_model=d_model, max_len=max_len)
x = torch.randn(batch_size, seq_len, d_model)
output = pe_layer(x)

print(f"输入形状: {x.shape}")          # [4, 64, 512]
print(f"输出形状: {output.shape}")     # [4, 64, 512]


输入形状: torch.Size([4, 64, 512])
输出形状: torch.Size([4, 64, 512])


- RoPE


(1)freq:
$$\Theta_j = \frac{1}{10000^{\frac{2j}{d}}} = 10000^{-\frac{2j}{d}}$$

旋转变化公式：
$$f(x, m) = \begin{pmatrix} \cos m\theta & -\sin m\theta \\ \sin m\theta & \cos m\theta \end{pmatrix} \begin{pmatrix} x_1 \\ x_2 \end{pmatrix}$$

展开：
$$\begin{pmatrix} x_1 \cos m\theta - x_2 \sin m\theta \\ x_1 \sin m\theta + x_2 \cos m\theta \end{pmatrix}$$

拆开:
$$\begin{pmatrix} x_1' \\ x_2' \end{pmatrix} = \underbrace{\begin{pmatrix} x_1 \\ x_2 \end{pmatrix} \cdot \cos \theta}_{\text{第一部分}} + \underbrace{\begin{pmatrix} -x_2 \\ x_1 \end{pmatrix} \cdot \sin \theta}_{\text{第二部分}}$$

(2)因此需要rotate_half：

$$\begin{pmatrix} -x_2 \\ x_1 \end{pmatrix}$$

(3)应用公式：
$$\text{RoPE}(x) = x \otimes \cos(m\theta) + \text{Rotate}(x) \otimes \sin(m\theta)$$

In [2]:
import torch


# 1. freqs： cos 和 sin 预计算
# 2. apply_rope

def precompute_freqs_cis(dim: int, end: int, theta: float = 10000.0):
   
    # 1. theta_i = 10000^(-2i/d)
    freqs = 1.0 / (theta ** (torch.arange(0, dim, 2)[: (dim // 2)].float() / dim))
    
    # 2. [0, 1, ..., max_len]
    t = torch.arange(end, device=freqs.device, dtype=torch.float32)
    
    # 3.类似register_buffer  : (end, dim/2)
    freqs = torch.outer(t, freqs)
    
    # 4. cos 和 sin
    freqs_cos = torch.cos(freqs)  # shape: (end, dim/2)
    freqs_sin = torch.sin(freqs)  # shape: (end, dim/2)
    
    return freqs_cos, freqs_sin

def reshape_for_broadcast(freqs_cis: torch.Tensor, x: torch.Tensor):

    ndim = x.ndim
    assert 0 <= 1 < ndim
    assert freqs_cis.shape == (x.shape[1], x.shape[-1])
    
    # freqs (max_len, dim/2) -> (1, max_len, 1, dim/2)
    shape = [d if i == 1 or i == ndim - 1 else 1 for i, d in enumerate(x.shape)]
    return freqs_cis.view(*shape)

def apply_rotary_emb(xq: torch.Tensor, xk: torch.Tensor, freqs_cos: torch.Tensor, freqs_sin: torch.Tensor):

    
    # 1. 旋转矩阵展开[x1,x2]与[-x2,x1]
    def rotate_half(x):
        x1 = x[..., : x.shape[-1] // 2] 
        x2 = x[..., x.shape[-1] // 2 :]
        return torch.cat((-x2, x1), dim=-1)
    
    # 2. reshape 以便广播

    freqs_cos = reshape_for_broadcast(freqs_cos, xq)
    freqs_sin = reshape_for_broadcast(freqs_sin, xq)

    # 3. 
    # rotate_half 计算
    # x_rotated = x * cos + rotate_half(x) * sin
    xq_out = (xq * freqs_cos) + (rotate_half(xq) * freqs_sin)
    xk_out = (xk * freqs_cos) + (rotate_half(xk) * freqs_sin)
    
    return xq_out, xk_out


dim = 64
seq_len = 10
bsz = 2
num_heads = 4

# 1. register_buffer  取出sin cos
cos, sin = precompute_freqs_cis(dim, seq_len) # (end, dim/2)


cos = torch.cat([cos, cos], dim=-1)
sin = torch.cat([sin, sin], dim=-1)

# input
xq = torch.randn(bsz, seq_len, num_heads, dim)
xk = torch.randn(bsz, seq_len, num_heads, dim)

# RoPE
xq_rope, xk_rope = apply_rotary_emb(xq, xk, cos, sin)

print("Output shape:", xq_rope.shape)

Output shape: torch.Size([2, 10, 4, 64])


# 1.Attention

- self attention

$$\text{Attention}(Q, K, V) = \text{Softmax}\left(\frac{Q K^T}{\sqrt{d_k}}\right) V$$

- Tensor.masked_fill(mask, value) 。其中mask需要为bool
-  torch.where(条件，mask==True 赋什么值，不满足条件的值)

In [3]:
import torch

scores = torch.randn(4, 4)
print(scores)

causal_mask = torch.ones(4, 4).triu(diagonal=1).bool()
# mask_1 = torch.ones(4, 4).tril(0).bool() # 主对角线以及以下【需要做成bool形式】
# mask_2 = torch.ones(4, 4).tril(-1) # 主对角线以下
# mask_3 = torch.ones(4, 4).tril(2) # 主对角线以上2行
print(causal_mask)
# print(mask_2) 
# print(mask_3)

# masked_fill(mask, value) 
scores  = scores.masked_fill(mask= causal_mask , value = 0)
print(scores)

# scores_where = torch.where(causal_mask, torch.tensor(0.0), scores) # True 位置设置为0
# print( scores_where)


# 这个方法比较正常
scores2 = torch.randn(4, 4)
att_weight = torch.ones(4, 4).tril() # 下三角矩阵 包含主对角线【default diagonal=0】
print(att_weight)
att_weight = scores2.masked_fill(att_weight == 0, float("-1e20"))

print(att_weight)




tensor([[-1.1165,  1.4655, -1.3211,  1.2021],
        [-2.6029, -0.5984, -0.2838, -0.5854],
        [-0.6428, -1.1741, -1.9348, -0.2217],
        [-0.5321, -1.2354,  1.3057,  1.7891]])
tensor([[False,  True,  True,  True],
        [False, False,  True,  True],
        [False, False, False,  True],
        [False, False, False, False]])
tensor([[-1.1165,  0.0000,  0.0000,  0.0000],
        [-2.6029, -0.5984,  0.0000,  0.0000],
        [-0.6428, -1.1741, -1.9348,  0.0000],
        [-0.5321, -1.2354,  1.3057,  1.7891]])
tensor([[1., 0., 0., 0.],
        [1., 1., 0., 0.],
        [1., 1., 1., 0.],
        [1., 1., 1., 1.]])
tensor([[ 9.0990e-01, -1.0000e+20, -1.0000e+20, -1.0000e+20],
        [-1.1419e+00, -2.1434e+00, -1.0000e+20, -1.0000e+20],
        [-1.6208e-01,  1.6470e+00, -1.4175e+00, -1.0000e+20],
        [-7.9996e-01,  1.0003e+00, -1.4757e+00,  3.1392e-01]])


In [2]:
import torch.nn as nn
import math
import torch

class SeAtten(nn.Module):
    def __init__(self, hidden_dim):
        super().__init__()
        
        self.hidden_dim = hidden_dim
        # q/k/v/o 
        self.q_proj = nn.Linear(hidden_dim, hidden_dim) 
        self.k_proj = nn.Linear(hidden_dim, hidden_dim) 
        self.v_proj = nn.Linear(hidden_dim, hidden_dim) 
        
        self.dropout = nn.Dropout(0.1) # 一般设置为 0.1
        
        self.o_proj = nn.Linear(hidden_dim, hidden_dim) 
    
    def forward(self, x, atten_mask = None):
        # input x: [batch,seq_len,hidden_dim]
        
        
        # q/k/v projection 
        q = self.q_proj(x)
        k = self.k_proj(x)
        v = self.v_proj(x)
        
        
        # attention value 
        ## dot : @ or torch.matmul
        # q [batch,seq_len,hidden_dim]
        # k [batch,seq_len,hidden_dim] =>【transpose】 [batch,hidden_dim,seq_len]
        
        # attention weight 
        ## scale : math.sqrt(hidden_dim)

        weight = (q @ k.transpose(-2,-1)) / math.sqrt(self.hidden_dim) # [batch,seq_len_q,seq_len_k]
        
        # attention mask
        if atten_mask is not None:
            # weight  = weight.where(atten_mask,float('-inf'),weight) # (mask, masked_value【mask==True】, score【mask==False】)
            weight = weight.masked_fill(atten_mask, float('-inf')) # (mask, masked_value【mask==True】)
        
        # apply softmax
        weight = torch.softmax(weight, dim=-1)  #[batch,seq_len_q,seq_len_k] 对k维度【dim = -1】做softmax
        
        # apply dropout
        weight = self.dropout(weight)

        # out projection
        
        # [batch,seq_len,seq_len] @ [batch,seq_len,hidden_dim] => [batch,seq_len,hidden_dim]
        out  = weight.matmul(v) # [batch,seq_len,hidden_dim]
        
        final = self.o_proj(out)
           

    
        return final
        
        
# test
## randn生成随机数张量（Tensor）
x = torch.randn(3, 4, 2) # [batch,seq_len,hidden_dim]
b = torch.tensor(
    [
        [1, 1, 1, 0],
        [1, 1, 0, 0],
        [1, 0, 0, 0],
    ]
)
print(b.shape)
mask = b.unsqueeze(dim=1).repeat(1, 4, 1).bool()  #  * time
print(mask.shape) # [3, 4, 4] 与atten value一致

net  = SeAtten(hidden_dim=2) # hidden_dim保持一致


output  = net(x,atten_mask = mask)
print(output)
print(output.shape) # [3, 4, 2]
        
        
        

torch.Size([3, 4])
torch.Size([3, 4, 4])
tensor([[[-0.8004, -0.6087],
         [-0.8004, -0.6087],
         [-0.8004, -0.6087],
         [-0.8004, -0.6087]],

        [[-0.5196, -0.4982],
         [-0.6512, -0.6370],
         [-0.6499, -0.6350],
         [-0.6488, -0.6331]],

        [[-0.2124, -0.2026],
         [-0.3469, -0.3512],
         [-0.3760, -0.3457],
         [-0.2301, -0.1825]]], grad_fn=<ViewBackward0>)
torch.Size([3, 4, 2])


- Multi-Head Attention

$$\text{Head}_i = \text{Attention}(\mathbf{Q} \mathbf{W}_i^Q, \mathbf{K} \mathbf{W}_i^K, \mathbf{V} \mathbf{W}_i^V)$$
$$\text{Attention}(\mathbf{Q}, \mathbf{K}, \mathbf{V}) = \text{Softmax}\left(\frac{\mathbf{Q} \mathbf{K}^T}{\sqrt{d_k}}\right) \mathbf{V}$$
$$\text{MultiHead}(\mathbf{Q}, \mathbf{K}, \mathbf{V}) = \text{Concat}(\text{Head}_1, \dots, \text{Head}_h) \mathbf{W}^O$$
- view()/contiguous()做连续内存的拆分与合并,premute()/transpose()做交换


In [ ]:
test  = torch.randn(2,3,4) # [b,s,hidden_dim]

batch_size, seq_len, _ = test.size()
print(batch_size, seq_len)
print(test.shape) # [2, 3, 4]


heads_num = 2
head_dim = 4 // heads_num

test = test.view(batch_size, seq_len, heads_num, head_dim)
print(test.shape) # [2, 3, 2, 2]

test1 = test
# permute 位置idx写在当前位置
test = test.permute(0,2,1,3)  # [b,heads,s,head_dim]
test1 = test1.transpose(2,1) # [2,1] [1,2]都都行

print(test.shape) # [2, 2, 3, 2]
print(test1.shape) # [2, 2, 3, 2]

2 3
torch.Size([2, 3, 4])
torch.Size([2, 3, 2, 2])
torch.Size([2, 2, 3, 2])
torch.Size([2, 2, 3, 2])


In [ ]:
import torch.nn as nn
import math


class MHA(nn.Module):
    def __init__(self, hidden_dim, nums_head):
        super().__init__()
        self.nums_head = nums_head
        self.hidden_dim = hidden_dim
        
        self.head_dim = hidden_dim // nums_head # MHA 计算才用
        
        # qkvo
        self.q_proj = nn.Linear(hidden_dim, hidden_dim)
        self.k_proj = nn.Linear(hidden_dim, hidden_dim)
        self.v_proj = nn.Linear(hidden_dim, hidden_dim)
        self.o_proj = nn.Linear(hidden_dim, hidden_dim)
        
        # dropout
        self.dropout = nn.Dropout(0.1)
        
    def forward(self, x, atten_mask = None):
        # input x: [batch,seq_len,hidden_dim]
        batch, seq_len, _ = x.size()
        
        
        # qkv projection [batch,seq_len,hidden_dim]
        q = self.q_proj(x) 
        k = self.k_proj(x)
        v = self.v_proj(x)
        
        # MHA x: 
        ## [batch,seq_len,hidden_dim]=>【view】[batch,seq_len,nums_head,head_dim]
        ## [batch,seq_len,nums_head,head_dim]=>【transpose/permute】 [batch,nums_head,seq_len,head_dim]
        
        # q_s  = q.view(batch, seq_len, self.nums_head, self.head_dim).permute(0, 2, 1, 3)
        q_s = q.view(batch, seq_len, self.nums_head, self.head_dim).transpose(1,2)
        k_s = k.view(batch, seq_len, self.nums_head, self.head_dim).transpose(1,2)
        v_s = v.view(batch, seq_len, self.nums_head, self.head_dim).transpose(1,2)
        
        # attention value ：q*k.T / sqrt(head_dim)
        # k_s [batch,nums_head,seq_len,head_dim]=> [batch,nums_head,head_dim,seq_len]
        
        weight = q_s @ k_s.transpose(-2,-1) / math.sqrt(self.head_dim) # [batch,nums_head,seq_len,seq_len]
        
        # attention mask
        if atten_mask is not None:
            weight = weight.masked_fill(atten_mask==0, float('-inf'))
            
        # softmax key dim
        weight = torch.softmax(weight, dim = -1) 
        
        # apply dropout
        weight = self.dropout(weight)
        
        # *v and concat
        ## [b,nums_h,s,s] @ [b,nums_h,s,d]=>[b,nums_h,s,d]
        out = weight @ v_s  # [b,nums_h,s,d]
        ## concat  [batch,nums_h,seq_len,head_dim]=>   [batch,seq_len,hidden_dim]
        out = out.transpose(1,2).contiguous().view(batch, seq_len, -1)
        
        
        final = self.o_proj(out)
        
        return final
    
# test

attention_mask = (
    torch.tensor(
        [
            [0, 1],
            [0, 0],
            [1, 0],
        ]
    )
    .unsqueeze(1)
    .unsqueeze(2)
    .expand(3, 8, 2, 2)
)

x = torch.rand(3, 2, 128)
net = MHA(128, 8)
net(x, attention_mask).shape
        

torch.Size([3, 2, 128])

- GQA


In [ ]:
# [batch,nums_kv_group,seq_len,head_dim]
k = torch.rand(3, 2, 8, 128)
nums_head = 16
nums_key_value_head = 2

print(k.shape) # [3, 2, 8, 128]
# repeat nums_kv_group => nums_head 次数
k = k.repeat_interleave(nums_head// nums_key_value_head, dim=1) # [3, 16, 8, 128]
print(k.shape)

torch.Size([3, 2, 8, 128])
torch.Size([3, 16, 8, 128])


In [ ]:
import torch.nn as nn
import math


class GQA(nn.Module):
    def __init__(self,hidden_dim,nums_head,nums_kv_group):
        super().__init__()
        
        # Q/G judge
        assert hidden_dim % nums_head ==0
        assert nums_head % nums_kv_group == 0
        
        self.hidden_dim = hidden_dim
        self.nums_head = nums_head

        # Q/G init
        self.head_dim = hidden_dim // nums_head # 计算kv dim
        self.nums_kv_group = nums_kv_group
        
        # qkvo【nums_head与nums_kv_group的区别】
        
        # q shape  (nums_head * head_dim)
        self.q_proj = nn.Linear(hidden_dim, hidden_dim) 
        # kv shape (nums_key_value_head * head_dim)
        self.k_proj = nn.Linear(hidden_dim,nums_kv_group*self.head_dim)
        self.v_proj = nn.Linear(hidden_dim,nums_kv_group*self.head_dim)
        
        self.o_proj = nn.Linear(hidden_dim, hidden_dim)
        
    
    def forward(self,x):
        # input x: [batch,seq_len,hidden_dim]
        batch, seq_len, _ = x.size()
        
        # qkv projection
        q = self.q_proj(x) # [batch,seq_len,hidden_dim]
        k = self.k_proj(x) # [batch,seq_len,nums_kv_group*self.head_dim]
        v = self.v_proj(x)     
        
        # mha split
        
        # q [batch,seq_len,hidden_dim]=>[batch,nums_head,seq_len,head_dim]
        q =  q.view(batch, seq_len, self.nums_head, self.head_dim).transpose(1,2)
        # kv [batch,seq_len,hidden_dim]=>[batch,nums_kv_group,seq_len,head_dim]
        k =  k.view(batch, seq_len, self.nums_kv_group, self.head_dim).transpose(1,2)
        v =  v.view(batch, seq_len, self.nums_kv_group, self.head_dim).transpose(1,2)
                      
        # kv repeat nums_head//nums_kv_group times dim = 1
        #  [batch,nums_kv_group,seq_len,head_dim] => [batch,nums_head,seq_len,head_dim]
        k = k.repeat_interleave(self.nums_head // self.nums_kv_group, dim = 1)
        v = v.repeat_interleave(self.nums_head // self.nums_kv_group, dim = 1)
        
        # weight
        weight = (q @ k.transpose(-2,-1)) / math.sqrt(self.head_dim) # [batch,nums_head,seq_len,seq_len]
        weight = torch.softmax(weight, dim = -1)
        
        # @v  
        #  [batch,nums_head,seq_len,seq_len] @ [batch,nums_head,seq_len,head_dim] => [batch,nums_head,seq_len,head_dim]
        out  = weight @ v #  [batch,nums_head,seq_len,head_dim]
        
        # concat
        # [batch,nums_head,seq_len,head_dim] => [batch,seq_len,nums_head,head_dim]  => [batch,seq_len,hidden_dim] 
        
        out = out.transpose(1,2).contiguous().view(batch, seq_len, -1)
        
        # o proj
        final = self.o_proj(out)
        
        return final
    
    
# test
x = torch.rand(3, 2, 128)
net = GQA(hidden_dim=128, nums_head=16, nums_kv_group=2)
out = net(x)
print(out.shape) # [3, 2, 128]

torch.Size([3, 2, 128])


- MQA : GQA的nums_kv_group ==1

# 2.Norm相关

- layernorm

均值 ($\mu$)：$$\mu = \frac{1}{\mathbf{H}} \sum_{i=1}^{\mathbf{H}} x_i$$  
方差 ($\sigma^2$)：$$\sigma^2 = \frac{1}{\mathbf{H}} \sum_{i=1}^{\mathbf{H}} (x_i - \mu)^2$$
公式：
$$\mathbf{y} = \frac{\mathbf{x} - \mu}{\sqrt{\sigma^2 + \epsilon}} \cdot \gamma + \beta$$

In [ ]:
class LayerNorm():
    def __init__():
        pass
    
    def forward():
        pass

In [ ]:
import torch
import torch.nn as nn

class LayerNorm(nn.Module):
    def __init__(self, h:int, eps:float=1e-8):
        super().__init__()
        self.h  = h                  # normalized_shape 通常是 [hidden_dim]
        self.gamma = nn.Parameter(torch.ones(self.h))
        self.beta = nn.Parameter(torch.zeros(self.h))
        self.eps = eps
        
    def _norm(self,x):
        # 1. mean
        mean = x.mean(dim=-1, keepdim=True)
        # 2. variance
        var = x.var(dim=-1, keepdim=True, unbiased=False)
        
        # 3. norm item
        return (x - mean) * torch.rsqrt(var + self.eps)
        

    def forward(self, x):
        # x [batch, seq_len, hidden_dim]
     
        out = self._norm(x.float()).type_as(x)
        
        return self.gamma * out + self.beta

# 测试代码
x = torch.randn(2, 3, 4)  # [batch, seq_len, hidden_dim]
ln = LayerNorm(4)
out = ln(x)

print("Input shape:", x.shape)
print("Output shape:", out.shape)

Input shape: torch.Size([2, 3, 4])
Output shape: torch.Size([2, 3, 4])


- RMSNorm

均方根 ($\text{RMS}$)：
    
$$\text{RMS}(\mathbf{x}) = \sqrt{\frac{1}{\mathbf{H}} \sum_{i=1}^{\mathbf{H}} x_i^2}$$

公式

$$y_{i}=\frac{x_{i}}{\sqrt{\frac{1}{n} \sum_{i=1}^{n}\left(x_{i}\right)^{2}+\epsilon}} * \gamma$$

In [ ]:
import torch

# 平方
a = 2**2
print(a)
a2 = pow(a,2)
print(a2)

# 开方求倒数
t = torch.tensor(4.0)
print(t)
a  =  torch.rsqrt(t)
print(a)


# mean
t2  = torch.ones(3,4)
mean_val1 = t2.mean(dim=1, keepdim=True) # 按列方向求平均,保持列结构
print(mean_val1, mean_val1.shape) # [3,1]
t2  = torch.ones(3,4)
mean_val2 = t2.mean(dim=1, keepdim=False) # 按列方向求平均，无列结构
print(mean_val2, mean_val2.shape) # [3,]


t2  = torch.ones(3,4)
mean_val1 = t2.mean(dim=0, keepdim=True) # 按行方向求平均，保持行结构
print(mean_val1, mean_val1.shape) # [1,4]

t2  = torch.ones(3,4)
mean_val1 = t2.mean() # 所有元素求平均
print(mean_val1, mean_val1.shape) # 标量

4
16
tensor(4.)
tensor(0.5000)
tensor([[1.],
        [1.],
        [1.]]) torch.Size([3, 1])
tensor([1., 1., 1.]) torch.Size([3])
tensor([[1., 1., 1., 1.]]) torch.Size([1, 4])
tensor(1.) torch.Size([])


In [ ]:
# * and @

import torch


X = torch.tensor([1.0, 2.0, 3.0, 4.0]) 
print(X.shape)
gamma = torch.tensor([10.0, 1.0, 2.0, 0.5]) 

# [1,4] * [1,4] => [1,4] 逐元素乘法
Y_element_wise = X * gamma
print(gamma.shape,Y_element_wise.shape)

# [1,4] @ [4,1] => [1,1]
gamma_matrix = gamma.view(4, 1) 
Z_matrix = X @ gamma_matrix
print(gamma_matrix.shape, Z_matrix.shape)



torch.Size([4])
torch.Size([4]) torch.Size([4])
torch.Size([4, 1]) torch.Size([1])


In [ ]:
import torch
import torch.nn as nn

class RMSNorm(nn.Module):
    def __init__(self,h:int,eps:float=1e-8):
        super().__init__()
        self.h = h
        self.eps = eps
        self.weight = nn.Parameter(torch.ones(h))# learnable parameter,对每个token的每个hidden_dim进行scale
        
    def _norm(self,x):
        # x: [batch,seq_len,hidden_dim]
        # pow => mean得rms => +eps => rsqrt
        
        return x * torch.rsqrt((x**2).mean(dim = -1, keepdim = True) + self.eps)

    def forward(self,x):
        # x.float() 保证计算精度
        # type_as(x) 恢复原始数据类型
        out = self._norm(x.float()).type_as(x)
        return out * self.weight
    
x = torch.randn(2,3,4)
layer = RMSNorm(h=4)

out = layer(x)
print(out)
print(out.shape)




tensor([[[-1.6832, -0.5923,  0.8886,  0.1621],
         [-0.5696, -0.1878, -1.3380,  1.3601],
         [ 0.5076, -0.2491,  1.2554,  1.4506]],

        [[ 1.3700,  0.8199,  0.1846,  1.1903],
         [-0.2055,  0.5257, -0.6524, -1.8044],
         [-0.6674,  0.2872, -1.3462,  1.2883]]], grad_fn=<MulBackward0>)
torch.Size([2, 3, 4])


# 3.FNN相关
- Transformer FNN


In [ ]:
import torch.nn as nn


class FeedForward(nn.Module):
    # 实际上就是 MLP
    def __init__(self, config):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(config.n_embd, 4 * config.n_embd),
             # 激活函数
             nn.ReLU(),  
            nn.Linear(4 * config.n_embd, config.n_embd),
            nn.Dropout(config.dropout)
        )
    
    def forward(self, x):
        return self.net(x)
    
x = torch.randn(2,3,4)
class Config:
    def __init__(self):
        self.n_embd = 4
        self.dropout = 0.1

net = FeedForward(Config())
out = net(x)
print(out.shape)

torch.Size([2, 3, 4])


- FNN

1. dim -> 4*dim- > GeLU -> dim

2. 添加了门控、逐元素相乘，act_func替换了 $\text{SiLU}$ 或 $\text{SwiGLU}$
refer minimind理解

In [24]:
import torch.nn as nn

class FeedForward(nn.Module):
    def __init__(self, dim):
        super().__init__()
        self.dim = dim
        
        self.net = nn.Sequential(
            nn.Linear(dim, 4*dim), # up
            nn.GELU(approximate='tanh'), # activation
            nn.Linear(4*dim, dim)
            
        )       
    
    def forward(self, x):
        
        out  = self.net(x)
        return out
    

- MoE
直接参考chaofa的了

In [ ]:
import torch.nn as nn
import torch
import torch.nn.functional as F


# MoE是mask router，token没稀疏化
# TKSA是根据router weight做稀疏化token【就是分类器那篇 】

# 1.FNN层【Expert】
class BasicExpert(nn.Module):
    def __init__(self, input_dim, output_dim):
        super().__init__()
        self.fc1 = nn.Linear(input_dim, 4 * input_dim)
        self.activation = nn.ReLU()
        self.fc2 = nn.Linear(4 * input_dim, output_dim)
    
    def forward(self, x):
        x = self.fc1(x)
        x = self.activation(x)
        x = self.fc2(x)
        return x


# 2.Router【我感觉这个就是后面我TKSA修改的代码】
# 主要参考自 mistral MOE 的实现
class MOERouter(nn.Module):
    def __init__(self, hidden_dim, expert_number, top_k):
        super().__init__()
        self.gate = nn.Linear(hidden_dim, expert_number)  ###！router！ 【TKSA就会模仿这个】
        self.expert_number = expert_number
        self.top_k = top_k
    
    def forward(self, hidden_states):
        # 计算路由logits
        # 对于每个token，计算它路由到每个专家的logits
        # hidden_states shape is (b * s, hidden_dim) 【！】
        router_logits = self.gate(hidden_states)  # shape is (b * s, expert_number)
        
        # 计算专家经过softmax之后的概率
        routing_probs = F.softmax(router_logits, dim=-1, dtype=torch.float)
        
        # 选择的idxa与weights
        router_weights, selected_experts = torch.topk(
            routing_probs, self.top_k, dim=-1
        )  # shape都是 (b * s, top_k)
        
        # 选择专家weight重新分配【！】
        router_weights = router_weights / router_weights.sum(dim=-1, keepdim=True)
        router_weights = router_weights.to(hidden_states.dtype)
        
        # 生成专家掩码
        expert_mask = F.one_hot(
            selected_experts,
            num_classes=self.expert_number
        )  # shape是 (b * s, top_k, expert_number)
        expert_mask = expert_mask.permute(2, 1, 0)  # (expert_number, top_k, b * s)
        
        return router_logits, router_weights, selected_experts, expert_mask


class MOEConfig:
    def __init__(
            self, 
            hidden_dim, 
            expert_number, 
            top_k, 
            shared_experts_number=2,
        ):
        self.hidden_dim = hidden_dim
        self.expert_number = expert_number
        self.top_k = top_k
        self.shared_experts_number = shared_experts_number

class SparseMOE(nn.Module):
    # 稀疏 MOE 模型，这里每一个 token 都会过 topk 个专家，得到对应token 的 hidden_embeddings
    def __init__(self, config):
        super().__init__()

        self.hidden_dim = config.hidden_dim

        self.expert_number = config.expert_number
        self.top_k = config.top_k

        self.experts = nn.ModuleList(
            [
                BasicExpert(self.hidden_dim, self.hidden_dim) for _ in range(self.expert_number) # 1. 初始化很多FNN层
            ]
        )

        self.router = MOERouter(self.hidden_dim, self.expert_number, self.top_k)
    
    def forward(self, x):
        # x shape is (b, s, hidden_dim)
        batch_size, seq_len, hidden_dim = x.size()

        # 因为后面是一个个token进行过专家处理，所以batch_size和seq_len要合并
        hidden_states = x.view(-1, hidden_dim) # shape is(b * s, hidden_dim)

        router_logits, router_weights, selected_experts_indices, expert_mask = self.router(hidden_states) # 得到weight
        # 其中 selected_experts_indices shape 是 (b * s, top_k)
        # 其中 expert_mask shape 是 (expert_number, top_k, b * s)
        
        final_hidden_states = torch.zeros(
            (batch_size * seq_len, hidden_dim),
            dtype=hidden_states.dtype,
            device=hidden_states.device
        )

        for expert_idx in range(self.expert_number):
            expert_layer = self.experts[expert_idx]
            # expert_mask[expert_idx] shape 是 (top_k, b * s)
            idx, top_x = torch.where(expert_mask[expert_idx]) 
            # idx 和 top_x 都是一维 tensor
            # idx 的值是 0 或 1, 表示这个 token 是作为当前专家的 top1 还是 top2
            # top_x 的值是 token 在 batch*seq_len 中的位置索引
            # 例如对于 batch_size=2, seq_len=4 的输入:
            # top_x 的值范围是 0-7, 表示在展平后的 8 个 token 中的位置
            # idx 的值是 0/1, 表示这个 token 把当前专家作为其 top1/top2 专家

            # hidden_states 的 shape 是 (b * s, hidden_dim)
            # 需要取到 top_x 对应的 hidden_states
            current_state = hidden_states.unsqueeze(
                0
            )[:, top_x, :].reshape(-1, hidden_dim) # （selected_token_number, hidden_dim）    ####！！！还没看懂

            # router_weight 的 shape 是 (b * s, top_k)
            current_hidden_states = expert_layer(
                current_state
            ) * router_weights[top_x, idx].unsqueeze(-1)  # （selected_token_number, 1） 这里有广播

            # 把当前专家的输出加到 final_hidden_states 中
            # 方式1 的写法性能更好，并且方式1容易出现
            final_hidden_states.index_add_(0, top_x, current_hidden_states.to(hidden_states.dtype))
            # 方式2
            # final_hidden_states[top_x] += current_hidden_states.to(hidden_states.dtype)
            # 方式2 的写法性能更差，并且方式2容易出现错误，+= 操作在处理重复索引时需要多次读写内存，可能会导致竞争条件

        # 把 final_hidden_states 还原到原来的 shape
        final_hidden_states = final_hidden_states.reshape(batch_size, seq_len, hidden_dim)

        return final_hidden_states, router_logits # shape 是 (b * s, expert_number)


def test_token_level_moe():
    x = torch.rand(2, 4, 16)
    config = MOEConfig(16, 2, 2)
    token_level_moe = SparseMOE(config)
    out = token_level_moe(x)
    print(out[0].shape, out[1].shape)


test_token_level_moe()

torch.Size([2, 4, 16]) torch.Size([8, 2])
